[![Fixel Algorithms](https://fixelalgorithms.co/images/CCExt.png)](https://fixelalgorithms.gitlab.io/)

# Image Processing with Python

## Gaussian Blur

> Notebook by:
> - Royi Avital RoyiAvital@fixelalgorithms.com

## Revision History

| Version | Date       | User        |Content / Changes                                                   |
|---------|------------|-------------|--------------------------------------------------------------------|
| 0.1.000 | 01/11/2024 | Royi Avital | First version                                                      |

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/FixelAlgorithmsTeam/FixelCourses/blob/master/ImageProcessingPython/0002SciKitImageBasics.ipynb)

In [ ]:
# Import Packages

# General Tools
import numpy as np
import scipy as sp
import pandas as pd

from numba import jit, njit

# Image Processing
import skimage as ski

# Machine Learning


# Miscellaneous
import math
import os
from platform import python_version
import random
import timeit

# Typing
from typing import Callable, List, Optional, Tuple

# Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# Jupyter
from IPython import get_ipython
from IPython.display import Image, display
from ipywidgets import Dropdown, FloatRangeSlider, FloatSlider, interact, IntSlider, Layout

## Notations

* <font color='red'>(**?**)</font> Question to answer interactively.
* <font color='blue'>(**!**)</font> Simple task to add code for the notebook.
* <font color='green'>(**@**)</font> Optional / Extra self practice.
* <font color='brown'>(**#**)</font> Note / Useful resource / Food for thought.

### Code Exercise

 - Single line fill

 ```python
 vallToFill = ???
 ```

 - Multi Line to Fill (At least one)

 ```python
 # You need to start writing
 ????
 ```

 - Section to Fill

```python
#===========================Fill This===========================#
# 1. Explanation about what to do.
# !! Remarks to follow / take under consideration.
mX = ???

???
#===============================================================#
```

In [ ]:
# Configuration
# %matplotlib inline

seedNum = 512
np.random.seed(seedNum)
random.seed(seedNum)

# sns.set_theme() #>! Apply SeaBorn theme

runInGoogleColab = 'google.colab' in str(get_ipython())

In [ ]:
# Constants



In [ ]:
# Fixel Algorithms Packages


In [ ]:
mX = np.random.rand(10)
np.ndim(mX)

In [ ]:
# General Auxiliary Functions

def PlotDft(mX: np.ndarray, 
            samplingFreq: float,                #<! Frequency [Hz]
            /, *, 
            applyDft: bool = True,
            numFreqBins: Optional[int] = None,
            singleSide: bool = True, 
            logScale: bool = True, 
            normalizeData: bool = False, 
            removeDc: bool = False,
            plotTitle: str = 'DFT',
            hA: Optional[plt.Axes] = None,
            ) -> plt.Axes:

    numDims = np.ndim(mX)
    if ((numDims > 2) or (numDims < 1)):
        raise ValueError('The input array must be a vector or a matrix')
    
    mXX = np.atleast_2d(mX)
    
    if np.iscomplexobj(mXX):
        singleSide = False
    
    if singleSide:
        fftFun     = np.fft.rfft
        fftFreqFun = np.fft.rfftfreq
    else:
        fftFun     = np.fft.fft
        fftFreqFun = np.fft.fftfreq
    
    numFreqBins = np.size(mXX, 1) if numFreqBins is None else numFreqBins
    
    vFftFreq = fftFreqFun(numFreqBins, 1 / samplingFreq)

    if applyDft:
        mK = np.abs(fftFun(mXX, numFreqBins, axis = 1))
    else:
        mK = np.abs(np.copy(mXX))
    
    yLabel = 'Amplitude'
    
    if logScale:
        mK = np.log1p(mK)
        yLabel = 'Amplitude [Log Scale]'
    
    if normalizeData:
        mK /= np.max(mK, axis = 1)[:, None]
    
    if removeDc:
        mK -= np.mean(mK, axis = 1)[:, None]

    hF, hA = plt.subplots(figsize = (10, 5))

    for ii, vK in enumerate(mK):
        hA.plot(vFftFreq, vK, linewidth = 2, label = f'Signal {(ii + 1):03d}')
    
    hA.set_title(plotTitle)
    hA.set_xlabel('Frequency [Hz]')
    hA.set_ylabel(yLabel)
    hA.legend()

    return hA



## The Discrete Fourier Transform (DFT)

The DFT, in the context of Linear Algebra, is basically a [change of basis](https://en.wikipedia.org/wiki/Change_of_basis).  
The DFT basis is composed by Harmonic signals.  
The basis allows analysis of data using an important concept, the Frequency.  

![](https://i.imgur.com/Qqlbz3R.png)

The DFT allows analyzing the spectrum of a signal:

1. The energy of a specific harmonic component.
2. The _Bandwidth_ of the signal (The support).

* <font color='brown'>(**#**)</font> The DFT can be derived as a Uniform Sampling of the [DTFT](https://en.wikipedia.org/wiki/Discrete-time_Fourier_transform).
* <font color='brown'>(**#**)</font> The [Nyquist Shannon Sampling Theorem](https://en.wikipedia.org/wiki/Nyquist%E2%80%93Shannon_sampling_theorem) is closely related to the Fourier Transform.

In [ ]:
# Parameters

numSamples = 100

## Generate Data

This section loads the image used to evaluate the results.  

In [ ]:
# Load / Generate Data

mX = np.random.randn(2, numSamples)

In [ ]:
# Display the Data

# hF, hA = plt.subplots(figsize = (8, 6))
# hA.imshow(mI, cmap = 'gray')
# hA.set_title(f'Input Image');

PlotDft(mX, 10);

## Gaussian Blur

Gaussian Blur is the most common filter in Image Processing.  
It is a superior LPF filter compared to the _Box Blur_ as its roll off is smoother and monotonic.  
It balances well between the quality of th output and computational burden.

![](https://i.imgur.com/lPusP2a.png)
<!-- ![](https://i.postimg.cc/YSgpVT9h/Gaussian-Blur.png) -->

There are many approaches to the implementation of the _Gaussian Blur_.  
Some of the more popular approaches are:

 - FIR Filter  
   The most naive implantation by creating a truncated support for the infinite Gaussian Kernel.  
   The complexity of this implantation, taking the advantage of the separability of the kernel, depends on its support length.  
   The quality gets better as the ratio between the support and the standard deviation ($\sigma$) gets larger.
 - IIR Filter  
   Approximating the FIR using an IIR filter. It makes teh implementation complexity independent of $\sigma$.
 - Approximation by Box Blur  
   Employing the [Central Limit Theorem](https://en.wikipedia.org/wiki/Central_limit_theorem) one could approximate the Gaussian Kernel by a repetitive application of the Box Blur.  
   Since there are implementations of the Box Blur which are independent of its support it allows such implementation for the Gaussian Filter as well.

This notebook demonstrates the connection between the support of the FIR implementation to the error.

* <font color='brown'>(**#**)</font> In depth analysis of methods to implement the Gaussian Blur is given in [Pascal Getreuer - A Survey of Gaussian Convolution Algorithms](https://www.ipol.im/pub/art/2013/87).
* <font color='brown'>(**#**)</font> The _Gaussian Kernel_ is used for generating other kernels as well: Derivative Kernels, Laplacian Kernels, etc...
* <font color='brown'>(**#**)</font> For Box Filter like implementation see [Efficient and Accurate Gaussian Image Filtering Using Running Sums](https://ieeexplore.ieee.org/document/6416657).

In [ ]:
# Interactive Visualization - Plot the Gaussian Kernel

hDispKernel = lambda σ: PlotGaussianKernel(σ, kernelRadiusVis)

σSlider = FloatSlider(min = 0.1, max = 7, step = 0.1, value = 2.0, layout = Layout(width = '30%'))
interact(hDispKernel, σ = σSlider);

plt.show();

### Error of the FIR Estimation

It can be shown that the Error of the FIR approximation of radius $r$ is given by:

$$ {\left\| \boldsymbol{k} - \boldsymbol{k}_{FIR} \right\|}_{1} \leq 2 \operatorname{erfc} \left( \frac{r}{ \sqrt{2 {\sigma}^{2}} } \right) $$

Where $\operatorname{erfc} \left( x \right) = 1 - \operatorname{erf} \left( x \right)$.

Which implies that given error tolerance $\epsilon$ the radius can be set:

$$ r = \lceil \sqrt{2} \operatorname{erfc}^{-1} \left( \frac{\epsilon}{2} \right) \sigma \rceil $$

For a given image the error is given by $\epsilon {\left\| \operatorname{Vec} \left( I \right) \right\|}_{\infty}$.




In [ ]:
# Error Optimized Gaussian Blur

#===========================Fill This===========================#
# 1. Implement the required Radius given an error tolerance as a function.
# 2. The function parameters are `σ`, `ε`.
# 3. Given the error bound the function calculates the support size `kernelRadius`.
# !! You may find `scipy.special.erfcinv()` useful.
#===============================================================#

def CalcGaussianKernelRadius( σ: float, ε: float = 1e-5 ) -> int:

    kernelRadius = math.ceil(math.sqrt(2) * sp.special.erfcinv(ε / 2) * σ)

    return kernelRadius

* <font color='green'>(**@**)</font> Given the function `scipy.special.erf()` implement its inverse using optimization o root finding function.

In [ ]:
# Display the Radius per Sigma and Error

vσ = np.linspace(0.1, 10, 1000)
vε = [1e-2, 1e-3, 1e-4, 1e-5, 1e-6]

hF, hA = plt.subplots(figsize = (7, 5))
for ε in vε:
    hA.plot(vσ, [CalcGaussianKernelRadius(σ, ε) for σ in vσ], lw = 2, label = f'ε = {ε}')

hA.set_xlabel('σ')
hA.set_ylabel('Kernel Radius')
hA.set_title('Gaussian Kernel Radius per Error Bound')
hA.legend();


* <font color='blue'>(**!**)</font> Convert the graph into Log Scale for `x` is the error bound and different values of `σ`.

In [ ]:
# Error Optimized Gaussian Blur

#===========================Fill This===========================#
# 1. Implement the Gaussian Blur as a function with a given error.
# 2. The function parameters are `mI`, `σ`, `ε`.
# 3. Given the error bound the function calculates the support and applies the Gaussian Blur.
# !! No need to implement the Gaussian Blur itself, you may use implementations by SciKit Image or SciPy.

def GaussianBlur( mI: np.ndarray, σ: float, ε: float = 1e-5 ) -> np.ndarray:
    """
    Applies a Gaussian Blur to an input image using a finite impulse response (FIR) approximation with a bounded error.

    The function applies a Gaussian filter with standard deviation `σ` to the input image `mI`. The FIR approximation
    of the Gaussian filter is truncated to ensure that the resulting blurred image meets the specified error bound `ε`.
    The kernel radius for the FIR approximation is calculated based on the error bound, which reduces the computation
    load by limiting the effective size of the Gaussian kernel.

    Parameters
    ----------
    mI : np.ndarray
        The input grayscale image represented as a 2D numpy array.
    σ : float
        The standard deviation of the Gaussian kernel, which determines the amount of blurring.
    ε : float, optional
        The error bound for the FIR approximation of the Gaussian kernel. A smaller value results in a larger
        kernel radius, increasing accuracy at the cost of additional computation. The default is `1e-5`.

    Returns
    -------
    mO : np.ndarray
        A 2D numpy array of the same shape as `mI`, containing the blurred image.

    Notes
    -----
    - **Gaussian Filter Approximation**: The Gaussian filter is theoretically infinite, but in practice, it is truncated
      to a finite radius to improve computational efficiency. The truncation radius is derived based on the specified
      error bound `ε`.
    - **Kernel Radius Calculation**: The truncation radius (`kernelRadius`) for the Gaussian kernel is calculated
      as:
      
          kernelRadius = ceil(sqrt(2) * erfcinv(ε / 2) * σ)

      where `erfinvc` is the complementary inverse error function. This radius ensures that the truncation error of the Gaussian kernel
      is within the specified error bound `ε`, meaning the probability of values outside the truncated region is less
      than or equal to `ε`.
    - **Gaussian Blur Application**: The `ski.filters.gaussian` function applies the Gaussian filter
      with the truncated kernel size defined by `truncate = kernelRadius / σ`.

    Example
    -------
    >>> import numpy as np
    >>> from skimage import data
    >>> mI = data.camera()  # Example grayscale image from skimage
    >>> mB = GaussianBlur(mI, σ = 2, ε = 1e-5)
    
    """

    kernelRadius = CalcGaussianKernelRadius(σ, ε)
    mO = ski.filters.gaussian(mI, sigma = σ, truncate = kernelRadius / σ) #<! Uses SciPy 

    return mO

#===============================================================#

In [ ]:
# Plot Results

def PlotResults( mI: np.ndarray, σ: float, ε: float ) -> plt.Figure:

    mORef   = GaussianBlur(mI, σ, 1e-6)
    mO      = GaussianBlur(mI, σ, ε)

    hF, vHa = plt.subplots(nrows = 1, ncols = 3, figsize = (16, 5))
    vHa = vHa.flat

    hA = vHa[0]
    hA.imshow(mI, cmap = 'gray', vmin = 0.0, vmax = 1.0)
    hA.set(xticks = [], yticks = [], title = 'Input Image')

    hA = vHa[1]
    hA.imshow(mORef, cmap = 'gray', vmin = 0.0, vmax = 1.0)
    hA.set(xticks = [], yticks = [], title = 'Reference Blur')

    hA = vHa[2]
    hA.imshow(mO, cmap = 'gray', vmin = 0.0, vmax = 1.0)
    hA.set(xticks = [], yticks = [], title = f'ε = {ε}, Error = {np.max(np.abs(mO - mORef))}')

    return None

In [ ]:
# Interactive Visualization - Plot the Gaussian Kernel

hPlotResults = lambda σ, ε: PlotResults(mI, σ, 10 ** (-ε))

σSlider = FloatSlider(min = 0.1, max = 4, step = 0.1, value = 2.0, description ='σ:', layout = Layout(width = '30%'))
εSlider = FloatSlider(min = 1.1, max = 5, step = 0.1, value = 2.0, description ='ε (Log Scale):', layout = Layout(width = '30%'))
interact(hPlotResults, σ = σSlider, ε = εSlider);

plt.show();

* <font color='brown'>(**#**)</font> A guideline is to keep the error bounded below 1 pixel value in `[0, 255]` scale.